<a href="https://colab.research.google.com/github/Adolfo-Hugo/Python-modelos-preditivos/blob/main/Prophet_acoes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**INSTALANDO BIBLIOTECA PROPHET**

In [ ]:
!pip install fbprophet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.0/64.0 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 61.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.9/47.9 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.8/91.8 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 60.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.7 MB/s eta 0:00:00
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for fbprophet
  Running set

**IMPORTANDO BIBLIOTECAS**

In [ ]:
!pip install yfinance

In [ ]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from prophet import Prophet
import datetime
from statsmodels.tsa.seasonal import seasonal_decompose
import yfinance as yf


**DOWNLOAD DAS ACOES DO MAGALU**

In [ ]:
from sklearn.metrics import mean_absolute_error

In [ ]:
magazine = yf.download("MGLU3.SA", start='2020-01-01')


[*********************100%***********************]  1 of 1 completed


**VISUALIZANDO DATASET 'ACOES'**

In [ ]:
magazine['Adj Close']

,Adj Close
Date,
2020-01-02,115.015053
2020-01-03,113.989174
2020-01-06,114.082436
2020-01-07,113.756027
2020-01-08,117.253342
...,...
2024-09-25,10.000000
2024-09-26,9.800000
2024-09-27,10.000000


**RENOMEANDO PARA MGLU**

In [ ]:
magazine = magazine.rename(columns = {'Adj Close':'MGLU'})
dataset_mglu = magazine['MGLU']
dataset_mglu.info()

<class 'pandas.core.series.Series'>
DatetimeIndex: 1184 entries, 2020-01-02 to 2024-10-01
Series name: MGLU
Non-Null Count  Dtype  
--------------  -----  
1184 non-null   float64
dtypes: float64(1)
memory usage: 18.5 KB


**EXPORTANDO O DF PARA CSV**

In [ ]:
magazine.to_csv('acoes_magazine.csv')

**IMPORTANDO O DF**

In [ ]:
magazine_df = pd.read_csv('acoes_magazine.csv')
magazine_df

,Date,Open,High,Low,Close,MGLU,Volume
0,2020-01-02,112.940292,115.348747,111.560692,115.348747,115.015053,3164766
1,2020-01-03,113.407951,115.348747,112.940292,114.319893,113.989174,2858647
2,2020-01-06,113.174126,115.301987,111.771141,114.413429,114.082436,2925106
3,2020-01-07,114.436813,115.044769,113.407951,114.086067,113.756027,2867329
4,2020-01-08,113.688553,117.733826,113.407951,117.593529,117.253342,4277365
...,...,...,...,...,...,...,...
1179,2024-09-25,10.270000,10.330000,9.840000,10.000000,10.000000,20478600
1180,2024-09-26,10.080000,10.330000,9.770000,9.800000,9.800000,22118500
1181,2024-09-27,9.800000,10.380000,9.800000,10.000000,10.000000,22961600
1182,2024-09-30,10.000000,10.060000,9.470000,9.700000,9.700000,25238600


**CONVERTENDO COLUNA DATA EM TIME E FILTRANDO O DATASET**

In [ ]:
dateparse = lambda dates: datetime.datetime.strptime(dates, '%Y-%m-%d')
dataset_mglu = pd.read_csv('acoes_magazine.csv', parse_dates=['Date'], index_col='Date',
                      date_parser = dateparse, usecols = ['Date', 'MGLU'])

<ipython-input-11-cbe5844a4584>:2: FutureWarning: The argument 'date_parser' is deprecated and will be removed in a future version. Please use 'date_format' instead, or read your data in as 'object' dtype and then call 'to_datetime'.
  dataset_mglu = pd.read_csv('acoes_magazine.csv', parse_dates=['Date'], index_col='Date',


**EXIBINDO DATASET**

In [ ]:
time_series = dataset_mglu['MGLU']
time_series

,MGLU
Date,
2020-01-02,115.015053
2020-01-03,113.989174
2020-01-06,114.082436
2020-01-07,113.756027
2020-01-08,117.253342
...,...
2024-09-25,10.000000
2024-09-26,9.800000
2024-09-27,10.000000


**VERIFICANDO VALORES MAX E MUN NO DF**

In [ ]:
time_series.index.min(), time_series.index.max()

(Timestamp('2020-01-02 00:00:00'), Timestamp('2024-10-01 00:00:00'))

In [ ]:
figura = px.line(title ='Historico por acoes MGLU')
figura.add_scatter(x = time_series.index, y = time_series.values, name = 'acoes')
figura.show()

In [ ]:
len(time_series)/2

592.0

In [ ]:
decomposicao = seasonal_decompose(time_series, period = 592)

In [ ]:
tendencia = decomposicao.trend
sazonal = decomposicao.seasonal
aleatorio = decomposicao.resid

**ANALISE DE TENDENCIA MAGALU**

In [ ]:
figura2 = px.line(title ='historico por acoes')
figura2.add_scatter(x = tendencia.index, y = tendencia.values, name = 'tendencia')

In [ ]:
magazine_df = magazine_df.rename(columns = {'Adj Close':'MGLU'})
dataset_magazine = magazine_df[['Date','MGLU']]
dataset_magazine


,Date,MGLU
0,2020-01-02,115.015053
1,2020-01-03,113.989174
2,2020-01-06,114.082436
3,2020-01-07,113.756027
4,2020-01-08,117.253342
...,...,...
1179,2024-09-25,10.000000
1180,2024-09-26,9.800000
1181,2024-09-27,10.000000
1182,2024-09-30,9.700000


**RENOMEANDO COLUNAS DATA FRAME PADRAO DO PROPHET**

In [ ]:
dataset_mglu = dataset_magazine[['Date','MGLU']].rename(columns = {'Date':'ds','MGLU':'y'}) #por padrao o prophet pede para que as colunas tenham esses nomes.
dataset_mglu.head()

,ds,y
0,2020-01-02,115.015053
1,2020-01-03,113.989174
2,2020-01-06,114.082436
3,2020-01-07,113.756027
4,2020-01-08,117.253342


**TREINANDO MODELO**

In [ ]:
dataset_mglu
modelo = Prophet()
modelo.fit(dataset_mglu)

INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmpqwut7zmw/vlfxwgts.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmpqwut7zmw/bdjd_z51.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=86140', 'data', 'file=/tmp/tmpqwut7zmw/vlfxwgts.json', 'init=/tmp/tmpqwut7zmw/bdjd_z51.json', 'output', 'file=/tmp/tmpqwut7zmw/prophet_model0b5eq3fk/prophet_model-20241002100121.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
10:01:21 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
10:01:23 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing


**REALIZANDO UMA PREVISAO DE 180 DIAS FUTURO**

In [ ]:
futuro = modelo.make_future_dataframe(periods=180) # PREVISAO DE 180 DIAS A FRENTE
previsoes = modelo.predict(futuro)

**VISUALIZANDO PREVISAO**

In [ ]:
previsoes

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
0,2020-01-02,113.778323,98.793941,121.815816,113.778323,113.778323,-3.668269,-3.668269,-3.668269,0.201133,0.201133,0.201133,-3.869402,-3.869402,-3.869402,0.0,0.0,0.0,110.110055
1,2020-01-03,113.796743,99.098140,121.511952,113.796743,113.796743,-3.784849,-3.784849,-3.784849,-0.007143,-0.007143,-0.007143,-3.777706,-3.777706,-3.777706,0.0,0.0,0.0,110.011894
2,2020-01-06,113.852003,98.671545,122.286996,113.852003,113.852003,-3.608223,-3.608223,-3.608223,-0.248403,-0.248403,-0.248403,-3.359819,-3.359819,-3.359819,0.0,0.0,0.0,110.243781
3,2020-01-07,113.870423,99.454007,122.751079,113.870423,113.870423,-2.957818,-2.957818,-2.957818,0.195326,0.195326,0.195326,-3.153144,-3.153144,-3.153144,0.0,0.0,0.0,110.912605
4,2020-01-08,113.888843,99.302777,122.816381,113.888843,113.888843,-2.786126,-2.786126,-2.786126,0.118910,0.118910,0.118910,-2.905035,-2.905035,-2.905035,0.0,0.0,0.0,111.102718
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1359,2025-03-26,1.339406,-36.430299,21.055615,-25.462009,27.445123,-9.675260,-9.675260,-9.675260,0.118910,0.118910,0.118910,-9.794170,-9.794170,-9.794170,0.0,0.0,0.0,-8.335854
1360,2025-03-27,1.298821,-36.863163,21.919171,-25.670116,27.730183,-9.557945,-9.557945,-9.557945,0.201133,0.201133,0.201133,-9.759078,-9.759078,-9.759078,0.0,0.0,0.0,-8.259124
1361,2025-03-28,1.258236,-35.724564,18.848245,-25.878223,27.824058,-9.652592,-9.652592,-9.652592,-0.007143,-0.007143,-0.007143,-9.645449,-9.645449,-9.645449,0.0,0.0,0.0,-8.394356
1362,2025-03-29,1.217652,-36.896302,21.213095,-26.079903,27.983929,-9.587240,-9.587240,-9.587240,-0.129911,-0.129911,-0.129911,-9.457329,-9.457329,-9.457329,0.0,0.0,0.0,-8.369589


In [ ]:
len(dataset_mglu), len(previsoes)
len(previsoes) - len(dataset_mglu)

180

**COMPARANDO O DATASET REAL VS DATASET DE PREVISÃO**

In [ ]:
dataset_mglu.head()

,ds,y
0,2020-01-02,115.015053
1,2020-01-03,113.989174
2,2020-01-06,114.082436
3,2020-01-07,113.756027
4,2020-01-08,117.253342


In [ ]:

previsoes[['ds','yhat']].head()

,ds,yhat
0,2020-01-02,110.110055
1,2020-01-03,110.011894
2,2020-01-06,110.243781
3,2020-01-07,110.912605
4,2020-01-08,111.102718


**VISUALIZANDO A PREVISAO DOS 180 DIAS FUTUROS**

In [ ]:
previsoes.tail(180)

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
1184,2024-10-02,8.441752,-2.907055,20.674998,8.441752,8.441752,1.086117,1.086117,1.086117,0.118910,0.118910,0.118910,0.967208,0.967208,0.967208,0.0,0.0,0.0,9.527869
1185,2024-10-03,8.401167,-1.752150,21.046771,8.401167,8.401167,1.412315,1.412315,1.412315,0.201133,0.201133,0.201133,1.211182,1.211182,1.211182,0.0,0.0,0.0,9.813483
1186,2024-10-04,8.360582,-1.364102,21.250240,8.360582,8.360582,1.451237,1.451237,1.451237,-0.007143,-0.007143,-0.007143,1.458380,1.458380,1.458380,0.0,0.0,0.0,9.811819
1187,2024-10-05,8.319997,-0.806966,21.868881,8.319997,8.319997,1.573005,1.573005,1.573005,-0.129911,-0.129911,-0.129911,1.702916,1.702916,1.702916,0.0,0.0,0.0,9.893003
1188,2024-10-06,8.279413,-1.562093,21.526854,8.279413,8.279413,1.809606,1.809606,1.809606,-0.129911,-0.129911,-0.129911,1.939517,1.939517,1.939517,0.0,0.0,0.0,10.089019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1359,2025-03-26,1.339406,-36.430299,21.055615,-25.462009,27.445123,-9.675260,-9.675260,-9.675260,0.118910,0.118910,0.118910,-9.794170,-9.794170,-9.794170,0.0,0.0,0.0,-8.335854
1360,2025-03-27,1.298821,-36.863163,21.919171,-25.670116,27.730183,-9.557945,-9.557945,-9.557945,0.201133,0.201133,0.201133,-9.759078,-9.759078,-9.759078,0.0,0.0,0.0,-8.259124
1361,2025-03-28,1.258236,-35.724564,18.848245,-25.878223,27.824058,-9.652592,-9.652592,-9.652592,-0.007143,-0.007143,-0.007143,-9.645449,-9.645449,-9.645449,0.0,0.0,0.0,-8.394356
1362,2025-03-29,1.217652,-36.896302,21.213095,-26.079903,27.983929,-9.587240,-9.587240,-9.587240,-0.129911,-0.129911,-0.129911,-9.457329,-9.457329,-9.457329,0.0,0.0,0.0,-8.369589


***VISUALIZANDO PREVISAO DAS ACOES ***

In [ ]:
previsoes[['ds','yhat']].tail(180) #ALEM DE 2024

,ds,yhat
1184,2024-10-02,9.527869
1185,2024-10-03,9.813483
1186,2024-10-04,9.811819
1187,2024-10-05,9.893003
1188,2024-10-06,10.089019
...,...,...
1359,2025-03-26,-8.335854
1360,2025-03-27,-8.259124
1361,2025-03-28,-8.394356
1362,2025-03-29,-8.369589


**GRAFICO DE PREVISÃO**

In [ ]:
from prophet.plot import plot_plotly
plot_plotly(modelo, previsoes) #azul previsão e preto valor real

In [ ]:
from prophet.plot import plot_components_plotly
plot_components_plotly(modelo, previsoes) #grafico de componentes de tendencia, anual, semanal

**AVALIACAO DO MODELO DE ML**

In [ ]:
pred = modelo.make_future_dataframe(periods=0)
previsoes = modelo.predict(pred)

In [ ]:
previsoes.shape

(1184, 19)

In [ ]:
previsoes.tail()

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,additive_terms,additive_terms_lower,additive_terms_upper,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,yhat
1179,2024-09-25,8.725846,-3.235924,20.657547,8.725846,8.725846,0.018563,0.018563,0.018563,0.118910,0.118910,0.118910,-0.100347,-0.100347,-0.100347,0.0,0.0,0.0,8.744409
1180,2024-09-26,8.685261,-2.794035,21.115209,8.685261,8.685261,0.139643,0.139643,0.139643,0.201133,0.201133,0.201133,-0.061490,-0.061490,-0.061490,0.0,0.0,0.0,8.824904
1181,2024-09-27,8.644676,-2.794540,19.828492,8.644676,8.644676,0.018038,0.018038,0.018038,-0.007143,-0.007143,-0.007143,0.025181,0.025181,0.025181,0.0,0.0,0.0,8.662714
1182,2024-09-30,8.522922,-3.048070,21.013908,8.522922,8.522922,0.266252,0.266252,0.266252,-0.248403,-0.248403,-0.248403,0.514655,0.514655,0.514655,0.0,0.0,0.0,8.789174
1183,2024-10-01,8.482337,-1.645030,20.411388,8.482337,8.482337,0.928136,0.928136,0.928136,0.195326,0.195326,0.195326,0.732811,0.732811,0.732811,0.0,0.0,0.0,9.410473


In [ ]:
previsoes = previsoes['yhat'].tail(180)


**MARGEM DE ERRO**

In [ ]:
mean_absolute_error(time_series.tail(180), previsoes)

4.197281253470019